In [2]:
from ase import Atoms
from ase.build import add_adsorbate, fcc111
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 08:44:01       11.689927       1.0797
BFGSLineSearch:    1[  2] 08:44:01       11.670814       0.4090
BFGSLineSearch:    2[  4] 08:44:01       11.625880       0.0409
Adsorption energy: 0.32351942231763786


In [3]:
#The Atoms object is a collection of atoms. Here is how to define a N2 molecule by directly specifying the position of two nitrogen atoms:
from ase import Atoms
d = 1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])

In [4]:
molecule

Atoms(symbols='N2', pbc=False)

In [19]:
#You can also build crystals using  the lattice module which returns Atoms objects corresponding to common crystal structures. 
#Let us make a Cu (111) surface:
from ase.build import fcc111
slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)

In [20]:
slab

Atoms(symbols='Cu32', pbc=[True, True, False], cell=[[10.210621920333747, 0.0, 0.0], [5.105310960166873, 8.842657971447272, 0.0], [0.0, 0.0, 22.08423447177455]], tags=...)

In [7]:
from ase.build import bcc110
slab2 = bcc110('Fe', size=(4,4,2), vacuum=10.0)

In [8]:
#In this overview we use the effective medium theory (EMT) calculator, as it is very fast and hence useful for getting started.
from ase.calculators.emt import EMT
slab.calc = EMT()
molecule.calc = EMT()

In [9]:
#calculate the total energies for the systems by using the get_potential_energy() method from the Atoms class:
e_slab = slab.get_potential_energy()
e_N2 = molecule.get_potential_energy()

In [10]:
e_slab

11.509056283569894

In [11]:
e_N2

0.440343573035614

In [12]:
# First add the adsorbate to the Cu slab, for example in the on-top position:
h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

In [13]:
#In order to speed up the relaxation, let us keep the Cu atoms fixed in the slab by using FixAtoms from the constraints module. 
#Only the N2 molecule is then allowed to relax to the equilibrium structure:
from ase.constraints import FixAtoms
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)

In [14]:
#Now attach the QuasiNewton minimizer to the system and save the trajectory file. 
#Run the minimizer with the convergence criteria that the force on all atoms should be less than some fmax:


from ase.optimize import QuasiNewton
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 08:44:01       11.689927       1.0797
BFGSLineSearch:    1[  2] 08:44:01       11.670814       0.4090
BFGSLineSearch:    2[  4] 08:44:01       11.625880       0.0409


True

In [15]:
#Writing the atomic positions to a file is done with the write() function:
from ase.io import write
write('slab.xyz', slab)

/home/ram/.local/lib/python3.10/site-packages/ase/io/extxyz.py:311: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [16]:
from ase.io import read
slab_from_file = read('slab.xyz')

In [17]:
slab_from_file

Atoms(symbols='Cu32N2', pbc=[True, True, False], cell=[[10.210621920333747, 0.0, 0.0], [5.105310960166873, 8.842657971447272, 0.0], [0.0, 0.0, 22.08423447177455]], tags=..., calculator=SinglePointCalculator(...))

In [29]:
#The simplest way to visualize the atoms is the view() function:
from ase.visualize import view
view(slab)

<Popen: returncode: None args: ['/usr/bin/python3', '-m', 'ase', 'gui', '-']>

In [26]:
#view(slab, viewer='VMD', data=array)
#Have to install VMD for this

In [37]:
#Let us look at the nitrogen molecule as an example of molecular dynamics with the VelocityVerlet algorithm. 
#We first create the VelocityVerlet object giving it the molecule and the time step for the integration of Newton’s law. 
#We then perform the dynamics by calling its run() method and giving it the number of steps to take:
from ase.md.verlet import VelocityVerlet
from ase import units
dyn = VelocityVerlet(molecule, dt=1.0 * units.fs)
for i in range(10):
    pot = molecule.get_potential_energy()
    kin = molecule.get_kinetic_energy()
    print('%2d: %.5f eV, %.5f eV, %.5f eV' % (i, pot + kin, pot, kin))
    dyn.run(steps=20)
#Didn't Understand

 0: 0.43843 eV, 0.27401 eV, 0.16442 eV
 1: 0.44028 eV, 0.43311 eV, 0.00717 eV
 2: 0.43813 eV, 0.27048 eV, 0.16765 eV
 3: 0.44002 eV, 0.40115 eV, 0.03887 eV
 4: 0.43918 eV, 0.32649 eV, 0.11269 eV
 5: 0.43987 eV, 0.38936 eV, 0.05051 eV
 6: 0.43880 eV, 0.32646 eV, 0.11234 eV
 7: 0.43877 eV, 0.32458 eV, 0.11420 eV
 8: 0.43988 eV, 0.39083 eV, 0.04905 eV
 9: 0.43916 eV, 0.32488 eV, 0.11428 eV
